In [1]:
from myEnv import MultiAgentEnv
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import rl_utils
import pickle
from tqdm import tqdm


class PolicyNetContinuous(torch.nn.Module):
    def __init__(self, state_dim, hidden_dim, action_dim):
        super(PolicyNetContinuous, self).__init__()
        self.fc1 = torch.nn.Linear(state_dim, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, hidden_dim)
        self.fc_mu = torch.nn.Linear(hidden_dim, action_dim)
        self.fc_std = torch.nn.Linear(hidden_dim, action_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        mu = torch.tanh(self.fc_mu(x))
        std = F.softplus(self.fc_std(x)) + 0.001
        return mu, std

    # def forward(self, x):
    #    x = F.tanh(x)
    #    x = self.fc1(x)
    #    mu = 2*self.fc_mu(x)
    #    std = F.softplus(self.fc_std(x))
    #    return mu, std


class ValueNet(torch.nn.Module):
    def __init__(self, state_dim, hidden_dim):
        super(ValueNet, self).__init__()
        hidden_dim = 2 * hidden_dim
        self.fc1 = torch.nn.Linear(state_dim, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = torch.nn.Linear(hidden_dim, hidden_dim)
        self.fc4 = torch.nn.Linear(hidden_dim, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))

        return self.fc4(x)


class PPOContinuous:
    ''' 处理连续动作的PPO算法 '''

    def __init__(self, state_dim, hidden_dim, action_dim, actor_lr, critic_lr,
                 lmbda, epochs, eps, gamma, device):
        self.actor = PolicyNetContinuous(state_dim, hidden_dim,
                                         action_dim).to(device)
        self.critic = ValueNet(state_dim * n_agents, hidden_dim).to(device)
        self.actor_optimizer = torch.optim.Adam(self.actor.parameters(),
                                                lr=actor_lr)
        self.critic_optimizer = torch.optim.Adam(self.critic.parameters(),
                                                 lr=critic_lr)
        self.gamma = gamma
        self.lmbda = lmbda
        self.epochs = epochs
        self.eps = eps
        self.device = device

    def take_action(self, state):
        state = torch.tensor(np.array([state]), dtype=torch.float).to(self.device)
        mu, sigma = self.actor(state)
        action_dist = torch.distributions.Normal(mu, sigma)
        action = action_dist.sample()
        #action = mu
        action = action.tolist()[0]
        return action


class MAPPO_agent:
    def __init__(self, state_dims, hidden_dim, action_dim, actor_lr, critic_lr,
                 lmbda, epochs, eps, gamma, device):
        self.agent = PPOContinuous(state_dims, hidden_dim, action_dim, actor_lr,
                                         critic_lr, lmbda, epochs, eps, gamma, device)
        self.gamma = gamma
        self.critic_criterion = torch.nn.MSELoss()
        self.device = device

    def take_action(self, states):
        actions = []
        for i in range(n_agents):
            action = self.agent.take_action(states[i])
            actions.append(action)
        return actions

    def update(self, transition_dict, i_agent):
        cur_agt = self.agent

        states = torch.tensor(transition_dict['states'],
                              dtype=torch.float).to(self.device)
        states_flatten = states.view(states.size(0), -1)
        states = states[:, i_agent, :]

        actions = torch.tensor(transition_dict['actions'],
                               dtype=torch.float).to(self.device)
        actions = actions[:, i_agent]

        rewards = torch.tensor(transition_dict['rewards'],
                               dtype=torch.float).to(self.device)
        rewards = rewards[:, i_agent].unsqueeze(-1)

        next_states = torch.tensor(transition_dict['next_states'],
                                   dtype=torch.float).to(self.device)
        next_states = next_states.view(next_states.size(0), -1)

        dones = torch.tensor(transition_dict['dones'],
                             dtype=torch.float).view(-1, 1).to(self.device)
        # rewards = (rewards + 8.0) / 8.0  # 和TRPO一样,对奖励进行修改,方便训练

        td_target = rewards + self.gamma * cur_agt.critic(next_states) * (1 - dones)
        td_delta = td_target - cur_agt.critic(states_flatten)
        advantage = rl_utils.compute_advantage(self.gamma, cur_agt.lmbda,
                                               td_delta.cpu()).to(self.device)
        mu, std = cur_agt.actor(states)
        action_dists = torch.distributions.Normal(mu.detach(), std.detach())
        # 动作是正态分布
        old_log_probs = action_dists.log_prob(actions)

        for _ in range(cur_agt.epochs):
            mu, std = cur_agt.actor(states)
            action_dists = torch.distributions.Normal(mu, std)
            log_probs = action_dists.log_prob(actions)
            ratio = torch.exp(log_probs - old_log_probs)
            surr1 = ratio * advantage
            surr2 = torch.clamp(ratio, 1 - cur_agt.eps, 1 + cur_agt.eps) * advantage
            actor_loss = torch.mean(-torch.min(surr1, surr2))
            critic_loss = torch.mean(
                F.mse_loss(cur_agt.critic(states_flatten), td_target.detach()))
            cur_agt.actor_optimizer.zero_grad()
            cur_agt.critic_optimizer.zero_grad()
            actor_loss.backward()
            critic_loss.backward()
            cur_agt.actor_optimizer.step()
            cur_agt.critic_optimizer.step()


actor_lr = 1e-5
critic_lr = 1e-4
num_episodes = 100
hidden_dim = 128
gamma = 0.95
lmbda = 0.9
epochs = 10
eps = 0.2
device = torch.device("cuda") if torch.cuda.is_available() else torch.device(
    "cpu")

env_name = 'myEnv'
env = MultiAgentEnv()
n_agents = env.n_agents - 1
torch.manual_seed(0)
state_dim = 4
action_dim = 1  # 连续动作空间

agent = MAPPO_agent(state_dim, hidden_dim, action_dim, actor_lr, critic_lr,
                    lmbda, epochs, eps, gamma, device)


# 加载模型
#agent.agents.critic.load_state_dict(torch.load('MAPPO/critic.pth'))
#agent.agents.actor.load_state_dict(torch.load('MAPPO/actor.pth'))

return_list = []
best_return = -300
for i in range(334):
    with tqdm(total=int(num_episodes / 10), desc='Iteration %d' % i) as pbar:
        for i_episode in range(int(num_episodes / 10)):
            episode_return = 0
            transition_dict = {'states': [], 'actions': [], 'next_states': [], 'rewards': [], 'dones': []}
            state = env.reset()
            done = False
            while not done:
                action = agent.take_action(state)
                next_state, reward, done, error, _ = env.step(action)
                transition_dict['states'].append(state)
                transition_dict['actions'].append(action)
                transition_dict['next_states'].append(next_state)
                transition_dict['rewards'].append(reward)
                transition_dict['dones'].append(done)
                state = next_state
                episode_return += error
            if episode_return > best_return:
                torch.save(agent.agent.critic.state_dict(), 'MAPPO/critic.pth')
                torch.save(agent.agent.actor.state_dict(), 'MAPPO/actor.pth')
                best_return = episode_return
            return_list.append(episode_return)
            for a_i in range(n_agents):
                agent.update(transition_dict, a_i)
            if (i_episode + 1) % 10 == 0:
                pbar.set_postfix({'episode': '%d' % (num_episodes / 10 * i + i_episode + 1),
                                  'return': '%.3f' % np.mean(return_list[-10:])})
            pbar.update(1)

with open('MAPPO_return_list.pkl', 'wb') as f:
    pickle.dump(return_list, f)

episodes_list = list(range(len(return_list)))
plt.plot(episodes_list, return_list)
plt.xlabel('Episodes')
plt.ylabel('Returns')
plt.title('PPO on {}'.format(env_name))
plt.show()

mv_return = rl_utils.moving_average(return_list, 21)
plt.plot(episodes_list, mv_return)
plt.xlabel('Episodes')
plt.ylabel('Returns')
plt.title('PPO on {}'.format(env_name))
plt.show()

state = env.reset()
# 加载模型
agent.agent.critic.load_state_dict(torch.load('MAPPO/critic.pth'))
agent.agent.actor.load_state_dict(torch.load('MAPPO/actor.pth'))

# 循环，直到环境结束
done = False
while not done:
    action = agent.take_action(state)  # 选择动作
    print("state:", state, "action", action)
    state, reward, done, _ = env.step(action)  # 执行动作

env.render()  # 显示环境

env.close()

Iteration 0:   0%|          | 0/10 [00:00<?, ?it/s]/root/rl_utils.py:91: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  return torch.tensor(advantage_list, dtype=torch.float)
Iteration 1:  40%|████      | 4/10 [00:05<00:08,  1.48s/it]


ValueError: Expected parameter loc (Tensor of shape (302, 1)) of distribution Normal(loc: torch.Size([302, 1]), scale: torch.Size([302, 1])) to satisfy the constraint Real(), but found invalid values:
tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
    